In [1]:
import tensorflow
import keras

from tensorflow.keras.models import Model , Sequential
from tensorflow.keras.layers import Dense , Flatten , Lambda , Input
from tensorflow.keras.applications.vgg19 import VGG19
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing import image
from glob import glob

In [2]:
IMAGE_SIZE = [224,224]

train_path = '/content/drive/My Drive/Colab Notebooks/chest_xray_images/train'
test_path = '/content/drive/My Drive/Colab Notebooks/chest_xray_images/test'

vgg19_model = VGG19(weights = 'imagenet',input_shape=IMAGE_SIZE + [3],include_top=False)

for layer in vgg19_model.layers:
  layer.trainable = False

folders = glob('/content/drive/My Drive/Colab Notebooks/chest_xray_images/train/*')

x = Flatten()(vgg19_model.output)

prediction = Dense(len(folders),activation = 'softmax')(x)

model = Model(inputs = vgg19_model.input , outputs = prediction)

model.summary()

model.compile(loss = 'categorical_crossentropy' , optimizer = 'adam', metrics = ['accuracy'])


80142336/80134624 [==============================] - 1s 0us/step
Model: "functional_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_______________________________________________________

In [3]:
from keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(rescale = 1./255,shear_range = 0.2,zoom_range = 0.3 , horizontal_flip = True)

test_datagen = ImageDataGenerator(rescale = 1./255)

training_set = train_datagen.flow_from_directory('/content/drive/My Drive/Colab Notebooks/chest_xray_images/train',target_size = (224,224),batch_size = 32 , class_mode = 'categorical')

test_set = test_datagen.flow_from_directory('/content/drive/My Drive/Colab Notebooks/chest_xray_images/test',target_size = (224,224),batch_size = 32 , class_mode = 'categorical')

model.fit_generator(training_set,validation_data= test_set,epochs = 5,steps_per_epoch = len(training_set),validation_steps =len(test_set))

Found 2077 images belonging to 2 classes.
Found 624 images belonging to 2 classes.
Instructions for updating:
Please use Model.fit, which supports generators.
Epoch 1/5
65/65 [==============================] - 1939s 30s/step - loss: 0.4895 - accuracy: 0.8247 - val_loss: 0.5023 - val_accuracy: 0.7901
Epoch 2/5
65/65 [==============================] - 74s 1s/step - loss: 0.2091 - accuracy: 0.9220 - val_loss: 0.5180 - val_accuracy: 0.7869
Epoch 3/5
65/65 [==============================] - 72s 1s/step - loss: 0.1793 - accuracy: 0.9360 - val_loss: 0.2889 - val_accuracy: 0.8846
Epoch 4/5
65/65 [==============================] - 72s 1s/step - loss: 0.1760 - accuracy: 0.9345 - val_loss: 0.4476 - val_accuracy: 0.8109
Epoch 5/5
65/65 [==============================] - 72s 1s/step - loss: 0.1507 - accuracy: 0.9456 - val_loss: 0.2695 - val_accuracy: 0.8910


In [5]:
model.save('chest_images_vgg19.h5')

In [6]:
from keras.preprocessing import image
from keras.applications.vgg19 import preprocess_input
from keras.models import load_model

import numpy as np

model = load_model('/content/chest_images_vgg19.h5')

img = image.load_img('/content/drive/My Drive/Colab Notebooks/chest_xray_images/val/NORMAL/NORMAL2-IM-1438-0001.jpeg',target_size = (224,224))
x = image.img_to_array(img)

x = np.expand_dims(x,axis= 0)

img_data = preprocess_input(x)

classes = model.predict(img_data)

In [7]:
classes

array([[1., 0.]], dtype=float32)

In [8]:
max = np.argmax(classes,axis = 1)

In [9]:
print(max)

[0]
